In [1]:
#load packages
import pandas as pd 
import numpy as np
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq

In [38]:
#load data and convert dates to datetime
fw = pd.read_csv("../Datasets/NEW/FW_with_airports.csv", parse_dates=['date'])
fw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23160 entries, 0 to 23159
Data columns (total 29 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   tail_number               23160 non-null  object        
 1   date                      23160 non-null  datetime64[ns]
 2   aircraft                  22966 non-null  object        
 3   origin                    23160 non-null  object        
 4   origin_location           23160 non-null  object        
 5   destination               23159 non-null  object        
 6   destination_location      23159 non-null  object        
 7   departure                 23160 non-null  object        
 8   arrival                   22734 non-null  object        
 9   duration                  22711 non-null  object        
 10  Owner                     23160 non-null  object        
 11  origin_Latitude           23001 non-null  float64       
 12  origin_Longitude  

In [43]:
fw_origin = fw.loc[:,['ICAO_code_origin', 'date', 'departure']]
fw_origin.head()

,ICAO_code_origin,date,departure
0,KXWA,2021-02-20,04:57PM CST
1,KBIS,2021-02-20,01:36PM CST
2,KMOT,2021-02-18,08:20AM CST
3,KXWA,2021-02-15,02:15PM CST
4,KMOT,2021-02-15,08:00AM CST


In [3]:
ICAO_origin = set(fw.ICAO_code_origin.dropna())
len(ICAO_origin)

1372

In [ ]:
ICAO_origin

In [107]:
url = 'http://www.ogimet.com/display_stations.php?lang=en&tipo=AND&isyn=&oaci=&nombre=&estado=&Send=Send'

In [108]:
# BeautifulSoup Magic
uClient = uReq(url)
page_html = uClient.read()
uClient.close()

In [109]:
page_soup = soup(page_html, 'html.parser')
codes = page_soup.findAll("a", {"onmouseout":" return nd();"})

In [112]:
ICAO = []

for i in range(len(codes)):
    ICAO += codes[i]

In [113]:
origin_ICAO = []

for i in range(len(codes)):
    if ICAO[i] in ICAO_origin:
        origin_ICAO += [ICAO[i]]

In [ ]:
url_2 = "http://www.ogimet.com/display_metars2.php?lang=en&lugar="
months = ['&tipo=ALL&ord=REV&nil=SI&fmt=html&ano=2021&mes=02&day=01&hora=23&anof=2021&mesf=02&dayf=28&horaf=23&minf=59&send=send', 
          '&tipo=ALL&ord=REV&nil=SI&fmt=html&ano=2021&mes=01&day=01&hora=23&anof=2021&mesf=01&dayf=31&horaf=23&minf=59&send=send',
         '&tipo=ALL&ord=REV&nil=SI&fmt=html&ano=2020&mes=12&day=01&hora=23&anof=2020&mesf=12&dayf=31&horaf=23&minf=59&send=send',
         '&tipo=ALL&ord=REV&nil=SI&fmt=html&ano=2020&mes=11&day=01&hora=23&anof=2020&mesf=11&dayf=30&horaf=23&minf=59&send=send']
date = []
METAR = []
ICAO_origin = []

for x in origin_ICAO:
    for j in months:
        url = url_2 + x + j
        uClient = uReq(url)
        page_html = uClient.read()
        uClient.close()
        page_soup = soup(page_html, 'html.parser')
    
        for i in range(len(page_soup.findAll("tr", {"bgcolor":["#F0F0D0",'white']}))):
            ICAO_origin += [x]
            date += page_soup.findAll("tr", {"bgcolor":["#F0F0D0",'white']})[i].findAll('td')[1]
            METAR += [page_soup.findAll("tr", {"bgcolor":["#F0F0D0",'white']})[i].findAll('td')[2].text]

In [ ]:
METAR